### Description:

AES-128 algorithm with a hardcoded key is available as service.

To perform encryption, the binary requires 16-bytes plaintext (in hex):

```
    ../wbcenc --ptext="010203040506070809000a0b0c0d0e1f"
```    
The result is printed to console:
```
    B2E0B38C3885C4F413E0E2C96AA59951 DB61E81D580E9097AD568BDB1031019C
```    
where:
```
    B2E0B38C3885C4F413E0E2C96AA59951 is a ciphertext
    
    DB61E81D580E9097AD568BDB1031019C is a trace (information left by a programmer)
```
### Tips:

A 'trace' is a side-channel information left by a programmer to debug the implementation.

In this example a trace is a 10th round input state of the AES-128 algorithm, i.e., a State before the Sbox operation in the last round.

### Task:

Your task is to find the Master key (round key 0) embedded into the binary. 

The master key is in the form of SCA{XXXXXXXXXXX}, where X is an ASCII printable symbol.

### Leakage illustration

<img src="support/Slide.png">

In [1]:
import numpy as np
import binascii
import string
import re
import socket

HOST = 'iict-mv330-sfa'
PORT = 4000

import sca_training
#----------------------------------------------------------------------------
# This function calls tested binary ../wbcenc either with a user-defined 
# plaintext (if plaintext satisfies all the requirements) or with a 
# predefined plaintext (the same as in the header above)
#
# INPUTS:
#     plaintext - a string of 32 symbols representing 16 hex bytes of ciphertext
#     verbose   - a flag to print values in the function call or not
# OUTPUTS:
#     output  - a raw binary output
#     ctext   - resulted ciphertext converted to numpy array of uint8
#     trace   - a trace associated with encryption process
#----------------------------------------------------------------------------
def binary_aes128_encrypt(plaintext, verbose=False):
    import subprocess
    pattern = '[0-9A-F]{32}'
    
    #Check the correctness of the plaintext
    if all(c in string.hexdigits for c in plaintext) and len(plaintext) == 32:
        
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            s.recv(1024)
            s.sendall(plaintext.encode())
            s.sendall(b'\n')

            output = s.recv(1024)
    #TODO: parse incoming data
        
    #Get the ciphertext (first 32 symbols) and the trace (last 32 symbols)
    result = re.findall(pattern, str(output))
    #print(result[0], result[1])
    
    #Transform the result into numpy array
    ctext = np.frombuffer(binascii.unhexlify(result[0]), dtype=np.uint8)
    trace = np.frombuffer(binascii.unhexlify(result[1]), dtype=np.uint8)

    if (verbose):
        print('Binary output:', output.strip())
        print('Ciphertext as numpy array:', ctext)
        print('Trace as numpy array:', trace)
    
    return output, ctext, trace


#----------------------------------------------------------------------------
# This function calls supportive library to compute a master key from
# the last round key.
#
# INPUTS:
#     last_round_key - a numpy array of 16 elements representing 16 hex bytes of
#                      the last round key, i.e., Round Key 10
# OUTPUTS:
#     key_schedule  - 11 round keys of 16 bytes each (key_schedule[0,0,:] is the
#                     master key
#----------------------------------------------------------------------------
# key_schedule = sca_training.inverse_key_expansion(last_round_key)

In [2]:
output, ctext, trace = binary_aes128_encrypt("010203040506070809000a0b0c0d0e1f", verbose=True)

Binary output: b'AA86E77B831376EB5177C26DDC0C1632 074EBE285954C25C0C3E71C7EFE30AB6'
Ciphertext as numpy array: [170 134 231 123 131  19 118 235  81 119 194 109 220  12  22  50]
Trace as numpy array: [  7  78 190  40  89  84 194  92  12  62 113 199 239 227  10 182]


### ATTACK CODE
Here you need to implement your attack which gives you the last round key.

Once the last round key is found - you need to compute the master key: this can be done with the function get_master_key()

### Programming tips
#### Numpy code
* np.arange(256).astype(np.uint8)
* np.bitwise_xor(m1, m2)
* np.where()

#### Code prepared for training
* sca_training.invSbox[sbox_out]
* sca_training.shift_rows(trace)

#### Various cod
* ''.join('{:02x}'.format(c) for c in int_array)

In [3]:
output, ctext, trace = binary_aes128_encrypt("010203040506070809000a0b0c0d0e1f", verbose=False)

#Define numpy structure to keep the recovered key
key = np.zeros((16), dtype=np.uint8)

key_cand = np.arange(256).astype(np.uint8)

#The global idea of side-channel attacks is to find a model, i.e., that takes known data, such as ciphertext bytes, and unknown key data (one key byte), and 
#predicts a leakage, which is measured by other means. 
#In this example the leakage is a raw 10th round input, so the model is equal to invShiftRows[invSbox[k ^ c[i,j]]], where k is a key byte 
# c is a ciphertext byte, i is an encrytion index, j is a byte index.
#In this example the model shall be strictly equal to the leakage when the key is correct, i.e. trace[v] = invShiftRows[invSbox[k ^ c[i,j]]].
#Since the invShiftRows operation is involved the indexes v and j are not necessary eqaul.
#This expression can be reqritten as: ShiftRows[trace][j] = invSbox[k ^ c[i,j]]
#When indexes v and j correspond to each other (due to the ShiftRows operation they are slightly shuffled), this expression for the correct key is correct,
#but for the wrong key this expression is strictly wrong (in this specific example). 
#Therefore, the idea of the attack is to compute the model invSbox[k ^ c[i,j]] or invShiftRows[invSbox[k ^ c[i,j]]] and compare the model with the associated
#leakage value.
#When the idexes v and j are selected correctly then in this example one encryption is enough to get the key.
#Shift the leakage samples so that they are aligned with the ciphertext bytes: ShiftRows[trace]
shifted_trace = sca_training.shift_rows(trace)

#Find all the key bytes one by one
for iByte in range(16):
    
    #Xor all the key candidates with the ciphertext byte: k ^ c[i,j]
    #k is a matrix of 256 elements (key candidate from 0 to 255)
    #i is equal to 0 (only one encrytion is needed)
    #j is iByte
    sbox_out = np.bitwise_xor(key_cand, ctext[iByte])
    
    #Apply inverse Sbox: invSbox[k ^ c[i, j]]
    sbox_in  = sca_training.invSbox[sbox_out]
    
    #Find the key (position) where the trace is equal to the sbox_in: ShiftRows[trace][j] == invSbox[k ^ c[i, j]]
    #There is only one key for which the model 'sbox_in' is equal to the byte of the correctly suffled trace in this particular example
    key[iByte] = np.where(sbox_in==shifted_trace[iByte])[0]

#Get master key using the provided binary
key_schedule = sca_training.inverse_key_expansion(key)
print('Master key in hex:', key_schedule[0,0,:])
print('Master key in ASCII:', binascii.unhexlify(''.join('{:02x}'.format(c) for c in key_schedule[0,0,:])))

Master key in hex: [ 72  69  73  71 123  83 105 109 112 108 101  32  65  69  83 125]
Master key in ASCII: b'HEIG{Simple AES}'


In [4]:
#Alternative solution without ShiftRows operation
import random
import numpy.matlib


#Create a 'database' of ciphertexts and corresponding leakages for several encryptions
num_enc = 3

traces = np.zeros((num_enc, 16), dtype=np.uint8)
ctexts = np.zeros((num_enc, 16), dtype=np.uint8)

for iEnc in range(num_enc):
    ptext = '%032x' % random.randrange(16**32)
    output, ctext, trace = binary_aes128_encrypt(ptext, verbose=False)
    ctexts[iEnc,:] = ctext
    traces[iEnc,:] = trace

del ptext
del output
del ctext
del trace

#Define numpy structure to keep the recovered key
key = np.zeros((16), dtype=np.uint8)

#Predefine all key candidates as a matrix num_enc x 256 
key_cand = np.matlib.repmat(np.arange(256).astype(np.uint8), num_enc,1)
  
#Find all the key bytes one by one
for iByte in range(16):
    
    #Xor all the key candidates with the ciphertext byte: k ^ c[i,j]
    #k is a matrix of 'num_enc x 256' elements (key candidate from 0 to 255 for each encryption)
    #i is the iterator from 0 to the number of encryptions num_enc, numpy can xor matrix with vector
    #j is iByte
    sbox_out = np.bitwise_xor(key_cand, ctexts[:, iByte].reshape(num_enc,1))

    #Apply inverse Sbox: invSbox[k ^ c[i, j]]
    sbox_in  = sca_training.invSbox[sbox_out]

    #Since we assume that the shuffling of the trace is unknown, we need to go throug all the samples and define the moment when key candidate provides
    #model vector 'sbox_in[:, key_cand]' which is strictly equal to the leakage vector 'traces[:,iSample]'. This can happen only once (for the correct key)
    #when the num_enc > 3.
    for iSample in range(16):
        #Compare each leakage vector "traces[:,iSample]" with all the models (associated for corresponding key candidate) in "sbox_in[: key_cand]".
        #Comparison will result in matrix of "True" and "False" with the size of num_enc x 256, where 256 is a number of key candidates
        #Now sum up column-wise elements in a comparison matrix. A number of hits (a sum) between a leakage at specific moment (iSample) and 
        #all key candidates is found.
        #The number of hits shows how many times our model corresponds (equal in this example) to the real leakage.
        #For the correct key this number of hits shall be equal to the number of encryptions, i.e., "num_enc"
        #For the wrong key this number of hits shall be less than "num_enc"
        #The correct number of hits, i.e., "num_enc", will occur only once when "iByte" corresponds to proper "iSample" and the key candidate is correct:
        #remember trace[:,iSample] == invSbox[k ^ c[:,iByte]]. Contrary to previous solution
        #invShiftRows is not needed here as all the iSamples will be tried.
        #"iByte" and "iSample" are linked via ShiftRows operation (but we are not using one iSample we try all of them, once the correct pair iByte and iSample
        #is met and the correct key is used, the number of hits will be equal to num_enc. In all other cases number of hits less than num_enc.
        #In all other cases, i.e., when "iByte" does not correspond to "iSample" or key candidate is wrong, the number of hits shall be less than "num_enc"
        #If "num_enc" is less than 3 then there might be a situation when a wrong key candidate gives the same number of hits as the correct key candidate
        #This can be mitigated by the increase of "num_enc" (plaintext shall be random)
        
        scores = np.sum(traces[:,iSample].reshape(num_enc,1) == sbox_in, axis=0)
        best_keys = np.where(scores == num_enc)[0]
        
        if len(best_keys) == 0:
            print('No key candidates remained for ciphertext byte', iByte, 'and sample index', iSample)
            
        if len(best_keys) == 1:
            print('Only one key candidate remained for ciphertext byte', iByte, 'and sample index', iSample)
            key[iByte] = best_keys[0]
        
        if len(best_keys) > 1:
            print('Several key candidates remained for ciphertext byte', iByte, 'and sample index', iSample)
            print(best_keys)
    
    print(key)

key_schedule = sca_training.inverse_key_expansion(key)
print(key_schedule.shape)

print('Master key in hex:', key_schedule[0,0,:])

print('Master key in ASCII:', binascii.unhexlify(''.join('{:02x}'.format(c) for c in key_schedule[0,0,:])))

Only one key candidate remained for ciphertext byte 0 and sample index 0
No key candidates remained for ciphertext byte 0 and sample index 1
No key candidates remained for ciphertext byte 0 and sample index 2
No key candidates remained for ciphertext byte 0 and sample index 3
No key candidates remained for ciphertext byte 0 and sample index 4
No key candidates remained for ciphertext byte 0 and sample index 5
No key candidates remained for ciphertext byte 0 and sample index 6
No key candidates remained for ciphertext byte 0 and sample index 7
No key candidates remained for ciphertext byte 0 and sample index 8
No key candidates remained for ciphertext byte 0 and sample index 9
No key candidates remained for ciphertext byte 0 and sample index 10
No key candidates remained for ciphertext byte 0 and sample index 11
No key candidates remained for ciphertext byte 0 and sample index 12
No key candidates remained for ciphertext byte 0 and sample index 13
No key candidates remained for cipherte

In [5]:
#Encrypt given input to check the correctness
from Crypto.Cipher import AES

aes = AES.new(b'HEIG{Simple AES}', AES.MODE_ECB)

ctext = aes.encrypt(binascii.unhexlify('010203040506070809000a0b0c0d0e1f'))

print(binascii.hexlify(ctext))

b'aa86e77b831376eb5177c26ddc0c1632'
